**Title**

**Introduction**:
• Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
• Clearly state the question you will try to answer with your project
• Identify and describe the dataset that will be used to answer the question

**Preliminary exploratory data analysis**:
• Demonstrate that the dataset can be read from the web into R 
• Clean and wrangle your data into a tidy format
• Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
• Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

**Methods**:
• Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
• Describe at least one way that you will visualize the results
• Expected outcomes and significance:
• What do you expect to find?
• What impact could such findings have?
• What future questions could this lead to?


In [5]:
library(tidyverse)
library(tidymodels)
library(repr)

In [11]:
#read the data
data <- read_csv("cleaned_5250.csv")

#clean the data
data <- data|>
    select(planet_type, name, radius_multiplier, radius_wrt) |>
    na.omit()
data

#types of planets 

    

Rows: 5250 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): name, planet_type, mass_wrt, radius_wrt, detection_method
dbl (8): distance, stellar_magnitude, discovery_year, mass_multiplier, radiu...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


planet_type,name,radius_multiplier,radius_wrt
<chr>,<chr>,<dbl>,<chr>
Gas Giant,11 Comae Berenices b,1.080,Jupiter
Gas Giant,11 Ursae Minoris b,1.090,Jupiter
Gas Giant,14 Andromedae b,1.150,Jupiter
Gas Giant,14 Herculis b,1.120,Jupiter
Gas Giant,16 Cygni B b,1.200,Jupiter
Gas Giant,17 Scorpii b,1.150,Jupiter
Gas Giant,18 Delphini b,1.110,Jupiter
Gas Giant,1RXS J160929.1-210524 b,1.664,Jupiter
Gas Giant,24 Bootis b,1.240,Jupiter
